# Position and orientation of rigid bodies

Define functions to work with rigid bodies

## Import functions needed

In [1]:
import numpy as np
import math

## Rotational matrix
Given an angle $\theta$ and a vector $r$, return a rotational matrix of the angle $\theta$ around the vector $r$.

In [2]:
def rotational(theta, r):
    # Separate x, y and z from vector r
    rx = r.item(0)
    ry = r.item(1)
    rz = r.item(2)
    
    # Calculate sin an cos of theta
    s = math.sin(theta)
    c = math.cos(theta)
    
    # Calculate rotational matrix
    rot = np.matrix([[rx*rx*(1-c)+c,    rx*ry*(1-c)-rz*s, rx*rz*(1-c)+ry*s],
                     [rx*ry*(1-c)+rz*s, ry*ry*(1-c)+c,    ry*rz*(1-c)-rx*s],
                     [rx*rz*(1-c)-ry*s, ry*rz*(1-c)+rx*s, rz*rz*(1-c)+c   ]])
    
    return rot

### Example
Rotational matrix of $\Pi$ around the vector $z$.

In [3]:
theta = math.pi
r = np.matrix([0,0,1])
R = rotational(theta, r)
print(R)

[[ -1.00000000e+00  -1.22464680e-16   0.00000000e+00]
 [  1.22464680e-16  -1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


## Check if $R$ is a rotational matrix
Return $True$ if matrix $R$ has the following properties to be rotational:
* Determinant equal to +1
* Its inverse is equal to its transpose

In [4]:
def is_rotational(R):
    # Check if R is ortogonal an
    if np.array_equal(np.transpose(R), np.linalg.inv(R)) and \
        np.linalg.det(R) == 1:
        return True

is_rotational(R)

True

## Inverse of rotational matrix
Given a rotational matrix $R$, return a unit vector $r$ and an angle $\theta$.

In [5]:
def inv_rotational(R):
    # Calculate R - R'
    RRt = R - np.transpose(R)
    
    # Calculate the trace the the matrix R
    trace = np.trace(R)
    
    # Calculate theta
    theta = math.atan2(math.sqrt(RRt.item(0,1)*RRt.item(0,1) +
                                 RRt.item(0,2)*RRt.item(0,2) +
                                 RRt.item(1,2)*RRt.item(1,2)), trace - 1)

    # If singular case
    if (math.sin(theta) < 0.0000001):
        if theta < 0.0000001:
            raise Exception('ERROR: theta needs to be different than 0!')
        elif abs(theta) == math.pi:
            RRt12 = np.sign(RRt.item(0,1))
            RRt13 = np.sign(RRt.item(0,2))
            raise Exception('TODO: correct sign, wrong result!')
            r = np.matrix([math.sqrt((R.item(0,0)+1)/2), RRt12*math.sqrt((R.item(1,1)+1)/2), RRt13*math.sqrt((R.item(2,2)+1)/2)])
    else:
        # Calculate the unit vector r
        r = 1/(2*math.sin(theta))*np.matrix([-RRt.item(1,2), RRt.item(0,2), -RRt.item(0,1)])
    
    return [theta, r]

### Example
Recover the angle $\theta$ and the vector $z$ given a rotational matrix $R$.

In [8]:
R = np.random.rand(3,3)
[theta, r] = inv_rotational(R)
print(theta)
print(r)

0.8439813022603257
[[ 0.32884278 -0.41827275  0.08648247]]


Recover the angle $\theta$ and the vector $z$ given a singular rotational matrix $R$, where the angle is equal to $\Pi$.

In [10]:
R = np.matrix([[-1,               0,               0],
               [ 0, -1/math.sqrt(2), -1/math.sqrt(2)],
               [ 0, -1/math.sqrt(2),  1/math.sqrt(2)]])

# First check to see if R is a rotational matrix
is_rotational(R)
[theta, r] = inv_rotational(R)
print(theta)
print(r)

Exception: TODO: correct sign, wrong result!

## Euler Angles

In [15]:
def euler_angles(phi, theta, psi):
    # Calculate cos of the angles
    cphi = math.cos(phi)
    ctheta = math.cos(theta)
    cpsi = math.cos(psi)
    
    # Calculate sin of the angles
    sphi = math.sin(phi)
    stheta = math.sin(theta)
    spsi = math.sin(psi)
    
    R = np.matrix([[cphi*cpsi-sphi*ctheta*spsi, -cphi*spsi-sphi*ctheta*cpsi, sphi*stheta],
                   [sphi*cpsi+cphi*ctheta*spsi, -sphi*spsi+cphi*ctheta*cpsi, -cphi*stheta],
                   [stheta*spsi, stheta*cpsi, ctheta]])
    
    return R

R = euler_angles(1.12, 2.25, 3.14)
print(R)

[[-0.43478138 -0.56611253  0.70034403]
 [-0.90053518  0.27225033 -0.33899283]
 [ 0.0012392  -0.77807221 -0.62817362]]


In [17]:
def inv_euler_angles(R):
    
    theta = math.atan2(math.sqrt(R.item(0,2)*R.item(0,2)+R.item(1,2)*R.item(1,2)), R.item(2,2))
    
    # If singular case
    if (math.sin(theta) < 0.0000001):
        raise Exception('ERROR: theta needs to be different than 0!')
    else:
        # Calculate phi and psi
        phi = math.atan2(R.item(0,2)/math.sin(theta), -R.item(1,2)/math.sin(theta))
        psi = math.atan2(R.item(2,0)/math.sin(theta), R.item(2,1)/math.sin(theta))
        
    return [phi, theta, psi]
    
[phi, theta, psi] = inv_euler_angles(R)
print(phi, theta, psi)

1.12 2.25 3.14


## Roll-Pitch-Yaw angles

In [22]:
def rpy_angles(psi, theta, phi):
    # Calculate cos of the angles
    cpsi = math.cos(psi)
    ctheta = math.cos(theta)
    cphi = math.cos(phi)
    
    # Calculate sin of the angles
    spsi = math.sin(psi)
    stheta = math.sin(theta)
    sphi = math.sin(phi)
    
    R = np.matrix([[cphi*ctheta, cphi*stheta*spsi-sphi*cpsi, cphi*stheta*cpsi+sphi*spsi],
                   [sphi*ctheta, sphi*stheta*spsi+cphi*cpsi, sphi*stheta*cpsi-cphi*spsi],
                   [-stheta, ctheta*spsi, ctheta*cpsi]])
    
    return R

R = rpy_angles(1.12, 2.25, 3.14)
print(R)

[[ 0.62817283 -0.70103703 -0.33755886]
 [-0.00100046 -0.43456649  0.9006392 ]
 [-0.7780732  -0.56541936 -0.27368422]]


In [23]:
def inv_rpy_angles(R):
    
    theta = math.atan2(-R.item(2,0),math.sqrt(R.item(2,1)*R.item(2,1)+R.item(2,2)*R.item(2,2)))
    
    # If singular case
    if (math.cos(theta) < 0.0000001):
        raise Exception('ERROR: cos(theta) needs to be different than 0!')
    else:
        # Calculate phi and psi
        phi = math.atan2(R.item(1,0)/math.cos(theta), R.item(0,0)/math.cos(theta))
        psi = math.atan2(R.item(2,1)/math.cos(theta), R.item(2,2)/math.cos(theta))
        
    return [psi, theta, phi]
    
[psi, theta, phi] = inv_rpy_angles(R)
print(psi, theta, phi)

-2.021592653589793 0.8915926535897932 -0.0015926535897931137
